# Exercise 4: Extended Unicycle Trajectory Tracking Control

Consider the extended kinematic model of a unicycle:
\begin{equation}
\dot{x} = v \cos(\theta), \quad \dot{y} = v \sin(\theta), \quad \dot{v}=a, \quad \dot{\theta} = \omega.
\end{equation}
where $(x,y)$ is the position, $\theta$ is the heading angle with respect to the x-axis, $v$ is the speed, and the control inputs are the acceleration $a$ and the angular velocity $\omega$. We define the state vector $\textbf{x} = (x, y, v, \theta)$.

In this exercise we will implement a differential flatness-based trajectory tracking controller for this robot.

In [ ]:
%load_ext autoreload
%autoreload 2
import typing as T
import numpy as np
from numpy import linalg
from matplotlib import pyplot as plt
from unicycle_utils import State, plot_unicycle_traj, simulate_unicycle

#### Exercise 4.1: Differentially Flat Trajectory Open-loop Control
Implement the `compute_traj_coeffs`, `compute_traj`, and `compute_controls` functions below to compute a differentially flat trajectory for the extended kinematic unicycle model above.

In [ ]:
def compute_traj_coeffs(s_0: State, s_f: State, t_f: float) -> np.ndarray:
    """
    Compute the flat trajectory coefficients to connect the initial to the final state.
    
    Parameters
    ----------
    s_0 (State) : the initial state
    s_f (State) : the final state
    t_f (float) : final time
        
    Returns
    -------
    coeffs (np.array shape [8]) : coefficients on the basis functions
    """
    ########## Code starts here ##########
    # Hint: Use the np.linalg.solve function.

    ########## Code ends here ##########
    return coeffs

def compute_traj(s_0: State, s_f: State, t_f: float, N: int) -> T.Tuple[np.ndarray, np.ndarray]:
    """
    Compute the trajectory from the basis function coefficients. Each point in time
    of the output trajectory is the vector [x, y, th, dx, dy, ddx, ddy].
    
    Parameters
    ----------
    s_0 (State) : the initial state
    s_f (State) : the final state
    t_f (float) : final_time
    N (int) : number of points
    
    Returns
    -------
    t (np.array shape [N]) : evenly spaced time points from 0 to t_f
    traj (np.array shape [N,7]) : N points along the trajectory, from t=0
                                  to t=t_f, evenly spaced in time
    """
    coeffs = compute_traj_coeffs(s_0, s_f, t_f)
    t = np.linspace(0, t_f, N) # generate evenly spaced points from 0 to t_f
    traj = np.zeros((N, 7))
    ########## Code starts here ##########

    ########## Code ends here ##########

    return t, traj

def compute_controls(traj: np.ndarray) -> T.Tuple[np.ndarray, np.ndarray]:
    """
    Convert the differentially flat trajectory into the controls [v, ω].
    
    Parameters
    ----------
    traj (np.array shape [N,7]) : differentially flat trajectory
    
    Returns
    -------
    ctrl (np.array shape [N,2]) : controls [v, ω] for each point of the trajectory
    """
    ########## Code starts here ##########

    ########## Code ends here ##########
    return np.column_stack((v, ω))

Now, run the code below for the provided initial and final states to see what the open loop differentially flat trajectory would look like for the extended kinematic unicycle.

In [ ]:
# Define constants
t_f  = 25 # final time (sec)
N = 501 # number of timesteps
s_0 = State(x=0, y=0, v=0.5, th=-np.pi/2)
s_f = State(x=5, y=5, v=0.5, th=-np.pi/2)

# Compute trajectory
t, traj = compute_traj(s_0, s_f, t_f, N)
ctrl = compute_controls(traj)

# Plot results
plot_unicycle_traj(t, (traj,), (ctrl,), ("nominal",))

Next, we will simulate how well we can actually execute these trajectories in the presense of noise. We will make use of the `simulate_unicycle` function to do the simulation of the same control sequence computed above.

In [ ]:
noise_scale = np.sqrt(0.2)
traj_ol, ctrl_ol = simulate_unicycle(s_0.x, s_0.y, s_0.th, t, open_loop_control=ctrl, noise_scale=noise_scale)

# Plot results
plot_unicycle_traj(t, (traj, traj_ol), (ctrl,), ("nominal", "ol"), (":", "-"), (0.7, 1))

#### Exercise 4.2: Closed-loop Tracking Control
In the class `TrajectoryTracker` implement the function `compute_control` to compute the controls for the differentially flat trajectory tracking controller from Example 3.2.2 in the text.

In [ ]:
V_PREV_THRES = 0.0001
class TrajectoryTracker:
    """ 
    Trajectory tracking controller using differential flatness 
    """
    def __init__(self, kpx: float, kpy: float, kdx: float, kdy: float,
                 v_max: float = 0.5, om_max: float = 1) -> None:
        self.kpx = kpx
        self.kpy = kpy
        self.kdx = kdx
        self.kdy = kdy

        self.v_max = v_max
        self.om_max = om_max

        self.coeffs = np.zeros(8) # Polynomial coefficients for x(t) and y(t) as
                                  # returned by the differential flatness code

    def reset(self) -> None:
        self.v_prev = 0.
        self.om_prev = 0.
        self.t_prev = 0.

    def load_traj(self, times: np.ndarray, traj: np.ndarray) -> None:
        """ 
        Loads in a new trajectory to follow, and resets the time 
        """
        self.reset()
        self.traj_times = times
        self.traj = traj

    def get_desired_state(self, t: float) -> T.Tuple[np.ndarray, np.ndarray, np.ndarray,
                                                     np.ndarray, np.ndarray, np.ndarray]:
        """
        Input:
            t: Current time
        Output:
            x_d, xd_d, xdd_d, y_d, yd_d, ydd_d: Desired state and derivatives
                at time t according to self.coeffs
        """
        x_d = np.interp(t, self.traj_times, self.traj[:,0])
        y_d = np.interp(t, self.traj_times, self.traj[:,1])
        xd_d = np.interp(t, self.traj_times, self.traj[:,3])
        yd_d = np.interp(t, self.traj_times, self.traj[:,4])
        xdd_d = np.interp(t, self.traj_times, self.traj[:,5])
        ydd_d = np.interp(t, self.traj_times, self.traj[:,6])

        return x_d, xd_d, xdd_d, y_d, yd_d, ydd_d

    def compute_control(self, x: float, y: float, th: float, t: float) -> T.Tuple[float, float]:
        """
        Inputs:
            x,y,th: Current state
            t: Current time
        Outputs:
            V, om: Control actions
        """

        dt = t - self.t_prev
        x_d, xd_d, xdd_d, y_d, yd_d, ydd_d = self.get_desired_state(t)

        ########## Code starts here ##########
        # Hint: Make sure to void singularity when v_prev is 0

        ########## Code ends here ##########

        # Apply control limits
        v = np.clip(v, -self.v_max, self.v_max)
        ω = np.clip(om, -self.om_max, self.om_max)

        # Save the commands that were applied and the time
        self.t_prev = t
        self.v_prev = v
        self.om_prev = ω

        return v, ω

Run the code below to see how the simulation differs for the unicycle with the same amount of noise as before, but this time applying the closed-loop tracking controller.

In [ ]:
controller = TrajectoryTracker(kpx=2, kpy=2, kdx=2, kdy=2, v_max=0.75, om_max=1)
controller.load_traj(t, traj)
traj_cl, ctrl_cl = simulate_unicycle(s_0.x, s_0.y, s_0.th, t, controller=controller, noise_scale=noise_scale)

# Plot results
plot_unicycle_traj(t, (traj, traj_cl), (ctrl, ctrl_cl), ("nominal", "closed-loop"), (":", "-"), (0.7, 1.0))